<a href="https://colab.research.google.com/github/suubkiim/Pytorch-practice/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Image processing
# Compose는 여러 transform 들을 chaining 합니다. 즉 여러 transform 진행합니다.
# 데이터를 torch Tensor로 바꾸고 Normalization을 위해 transform.Compose 를 사용합니다.
# transform 옵션 세부사항 정의 부분
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    #torch.nn.LeakyReLU(negative_slope=0.01, inplace=False)
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size), #(n_features, out_features, bias=True)
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
####################################quesiton####################################
# latent_size는 임의로 지정? 전처리로 만들어진 데이터의 크기부터 네트워크가    # 
# 정해지는 것인가?                                                             #
################################################################################
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy(BCE) loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)


####################################quesiton####################################
# denorm 이 무엇인지                                                           #
################################################################################
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()


0it [00:00, ?it/s]

9920512it [00:05, 1864046.17it/s]                             


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


32768it [00:00, 434198.28it/s]
  1%|          | 16384/1648877 [00:00<00:11, 144878.77it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


1654784it [00:00, 7170058.84it/s]                            
8192it [00:00, 181978.57it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw
Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


=========> 참고 링크 **[LeakyReLU](https://pytorch.org/docs/stable/nn.html#leakyrelu)**
>> **negative_slope** – Controls the angle of the negative slope. Default: 1e-2

>> **inplace** – can optionally do the operation in-place. Default: False (# inplace 하면 input으로 들어온 것 자체를 수정하겠다는 뜻. 메모리 usage가 좀 좋아짐. 하지만 input을 없앰./inplace=True means that it will modify the input directly, without allocating any additional output. It can sometimes slightly decrease the memory usage, but may not always be a valid operation (because the original input is destroyed). )

=========> 참고링크 **[nn.Linear](https://pytorch.org/docs/stable/nn.html#linear)**
>> **in_features** – size of each input sample

>> **out_features** – size of each output sample

>> **bias** – If set to False, the layer will not learn an additive bias. Default: True


=========> 참고링크 **[BCELoss](https://jaeyung1001.tistory.com/45)**
=========> 참고링크 **[BCELoss VS CrossEntropy](https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-7/06-multi_classification)**
>> 제일먼저 **CrossEntropy** 같은 경우에는 마지막 레이어 노드수가 2개 이상이여야 한다.

1개일 경우에는 사용이 안됨.



마지막 레이어가 노드수가 1개일 경우에는 보통 **Binary Classification**을 할때 사용될수가 있는데

이럴경우 **BCELoss**를 사용할때가 있다.



BCELoss함수를 사용할 경우에는 먼저 **마지막 레이어의 값이 0~1로 조정을 해줘야하기 때문에**

단순하게 마지막 레이어를 nn.Linear로 할때 out of range 에러가 뜬다.


따라서 **BCELoss함수**를 쓸땐 **마지막 레이어를 시그모이드함수를 적용**시켜줘야 한다.


---


![G/D 네트워크 구조](https://sthalles.github.io/assets/dcgan/GANs.png)


In [4]:

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

RuntimeError: ignored